<a href="https://colab.research.google.com/github/habibsifat/Dna2Vec/blob/master/Dna2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
from __future__ import print_function
__author__ = 'maxim'
import numpy as np
import pandas as pd
import gensim
from gensim.models import KeyedVectors
import string
from keras.layers.core import Dense, Activation, Dropout
from keras.callbacks import LambdaCallback
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Activation
from keras.models import Sequential
from keras.utils.data_utils import get_file
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [25]:
corpus_file = '/content/drive/My Drive/Colab Notebooks/BIO/Promoter/Dna2Vec/Sigma70/70ds44.txt'
print('\nPreparing the  sentences...')
sentences = []
with open(corpus_file, 'r', encoding="utf-8-sig") as f:
    for line in f:
        sentences.append(line.split())
        
print('Num sentences:', len(sentences))


Preparing the  sentences...
Num sentences: 3023


# **w2v**

In [0]:
!pip install gensim

In [26]:
from gensim.models import Word2Vec
print('\nTraining word2vec...')
word_model = Word2Vec(sentences, size=100, min_count=1, window=5, iter=10)
#pretrained_weights = word_model.wv.syn0 
#vocab_size, emdedding_size = pretrained_weights.shape
#print('Result embedding shape:', pretrained_weights.shape)
len(word_model.wv.vocab)
#Save partly trained model
#word_model.save('w2v_model.h5')


Training word2vec...


261

In [10]:
word_model.wv.vocab

{'A': <gensim.models.keyedvectors.Vocab at 0x7f8fbbe9e978>,
 'C': <gensim.models.keyedvectors.Vocab at 0x7f8fbbe9e908>,
 'G': <gensim.models.keyedvectors.Vocab at 0x7f8fa5c68e48>,
 'T': <gensim.models.keyedvectors.Vocab at 0x7f8f5bd8d358>,
 '\ufeff': <gensim.models.keyedvectors.Vocab at 0x7f8f5bd8d2b0>}

In [0]:
word_model.wv['\ufeff']

# **Train**

In [0]:
pretrained_weights = word_model.wv.syn0 
vocab_size, emdedding_size = pretrained_weights.shape
print('Result embedding shape:', pretrained_weights.shape)
print(vocab_size)
print(emdedding_size)

In [0]:
def word2idx(word):
  return word_model.wv.vocab[word].index
def idx2word(idx):
  return word_model.wv.index2word[idx]

In [0]:
max_sentence_len = 78
print('\nPreparing the data for LSTM...')
x = np.zeros([len(sentences), max_sentence_len], dtype=np.int32)
#y = np.zeros([len(sentences)], dtype=np.int32)
for i, sentence in enumerate(sentences):
  for t, word in enumerate(sentence[:-1]):
    x[i, t] = word2idx(word)
    #y[i] = word2idx(sentence[-1])
print('x shape:', x.shape)
#print('y shape:', y.shape)


Preparing the data for LSTM...
x shape: (5720, 78)


In [0]:
y = np.array([1]*2860 + [0]*2860)
print('y shape:', y.shape)

y shape: (5720,)


In [0]:
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(x,y,test_size=0.1,random_state=10)
#shape of train and test objects
print(X_train.shape)
print(X_test.shape)
# shape of new y objects
print(Y_train.shape)
print(Y_test.shape)

(5148, 78)
(572, 78)
(5148,)
(572,)


# LSTM

In [0]:
from keras import optimizers
print('\nTraining LSTM...')
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=emdedding_size, weights=[pretrained_weights]))
model.add(LSTM(units=512,return_sequences=True))
#model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(LSTM(units=512, return_sequences=False))
#model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(units=vocab_size))
model.add(Activation('softmax'))
optz = optimizers.Adam(lr=1, clipnorm=5,beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(optimizer=optz, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


Training LSTM...


In [0]:
history = model.fit(X_train, Y_train,batch_size=128,epochs=5,shuffle=True, validation_data=(X_test, Y_test))

# CNN

In [0]:
from keras.layers import Convolution1D, Dense, Dropout, Flatten
from keras.optimizers import Adam,RMSprop
from keras.callbacks import EarlyStopping
model = Sequential()
#model.add(Embedding(input_dim=vocab_size, output_dim=emdedding_size, weights=[pretrained_weights]))
#model.add(Embedding(4, 100, input_length=81))
model.add(Convolution1D(200, kernel_size=3, activation='relu', input_shape=(81, 1), name='m1_conv1'))
#model.add(Dropout(0.5, name='m1_drop1'))
model.add(Convolution1D(128, kernel_size=3, activation='relu', name='m1_conv2'))
#model.add(Dropout(0.5, name='m1_drop2'))
model.add(Convolution1D(64, kernel_size=3, activation='relu', name='m1_conv3'))
#model.add(Dropout(0.5, name='m1_drop3'))
model.add(Flatten(name='m1_flatten1'))
#model.add(Dropout(0.5, name='m1_drop4'))
model.add(Dense(64, activation='relu', name='m1_dense1'))
model.add(Dense(1, activation='softmax', name='m1_dense2'))

#model.compile(RMSprop(lr=.003), loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(Adam(lr=.003), loss='binary_crossentropy', metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', patience=16)
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=5, batch_size=32, callbacks=[es])

# 1-Mer

In [0]:
df = pd.read_csv('/content/promoter70.txt')
seqs = df.iloc[:, 0].tolist()

for i in range(len(seqs)):
  seqs[i] = seqs[i].upper()
  
with open('/content/drive/My Drive/Colab Notebooks/BIO/Promoter/Dna2Vec/Sigma70/70ds11.txt', 'w', encoding="utf-8-sig") as f:
  for j in seqs:
    for k in j:
      f.write(k)
      f.write(" ")
    f.write("\n")
f.close()

In [0]:
%pwd

'/content'

**this cell**

In [0]:
A4 = np.zeros([2860,81,100],dtype=np.int32)

for i in range(0,len(sentences)): 
  p = 0
  for j in sentences[i]:
    q = 0
    for k in word_model.wv[j]:
      r = 0
      A4[p][q][r] = k
      r=r+1
    q=q+1
  p=p+1

In [0]:
#second dimesion of the array 81 because there are total 81 1-mers
A2 = np.zeros([3023,79,100])

for current_sequence_number in range(0,len(sentences)):

  kmer_counter = 0
  current_sequence = sentences[current_sequence_number] 
  
  #here j contains a k-mer from current_sequence

  for kmer in current_sequence:
    
    A2[current_sequence_number,kmer_counter,:] = word_model.wv[kmer] 

    kmer_counter = kmer_counter + 1

In [0]:
np.save('/content/drive/My Drive/Colab Notebooks/BIO/Promoter/Dna2Vec/Sigma70/70A44.npy',A2)

In [0]:
A2[1][1]

In [0]:
word_model.wv['CGCG']

In [0]:
sentences[1][1]

# 2-Mer

In [0]:
df = pd.read_csv('/content/promoter70.txt')
seqs = df.iloc[:, 0].tolist()

for i in range(len(seqs)):
  seqs[i] = seqs[i].upper()
  
with open('/content/drive/My Drive/Colab Notebooks/BIO/Promoter/Dna2Vec/Sigma70/70ds22.txt', 'w', encoding="utf-8-sig") as f:#
  for j in seqs:
    for k in range(0,len(j)-1):
      #print(j[k],end='')
      #print(j[k+1],end=' ')
      f.write(j[k])
      f.write(j[k+1])
      f.write(" ")
    f.write("\n")
    #print()
f.close()

# **3-Mer**

In [0]:
df = pd.read_csv('/content/promoter70.txt')
seqs = df.iloc[:, 0].tolist()

for i in range(len(seqs)):
  seqs[i] = seqs[i].upper()
  
with open('/content/drive/My Drive/Colab Notebooks/BIO/Promoter/Dna2Vec/Sigma70/70ds33.txt', 'w', encoding="utf-8-sig") as f:#
  for j in seqs:
    for k in range(0,len(j)-2):
      #print(j[k],end='')
      #print(j[k+1],end=' ')
      f.write(j[k])
      f.write(j[k+1])
      f.write(j[k+2])
      f.write(" ")
    f.write("\n")
    #print()
f.close()

# **4-Mer**

In [0]:
df = pd.read_csv('/content/promoter70.txt')
seqs = df.iloc[:, 0].tolist()

for i in range(len(seqs)):
  seqs[i] = seqs[i].upper()
  
with open('/content/drive/My Drive/Colab Notebooks/BIO/Promoter/Dna2Vec/Sigma70/70ds44.txt', 'w', encoding="utf-8-sig") as f:#
  for j in seqs:
    for k in range(0,len(j)-3):
      #print(j[k],end='')
      #print(j[k+1],end=' ')
      f.write(j[k])
      f.write(j[k+1])
      f.write(j[k+2])
      f.write(j[k+3])
      f.write(" ")
    f.write("\n")
    #print()
f.close()

# **MODEL**

In [29]:
X1 = np.load('/content/drive/My Drive/Colab Notebooks/BIO/Promoter/Dna2Vec/Sigma70/70A11.npy')
X2 = np.load('/content/drive/My Drive/Colab Notebooks/BIO/Promoter/Dna2Vec/Sigma70/70A22.npy')
X3 = np.load('/content/drive/My Drive/Colab Notebooks/BIO/Promoter/Dna2Vec/Sigma70/70A33.npy')
X4 = np.load('/content/drive/My Drive/Colab Notebooks/BIO/Promoter/Dna2Vec/Sigma70/70A44.npy')
print('X shape:', X1.shape)
y = np.array([1]*1694 + [0]*1329)
print('y shape:', y.shape)

X shape: (3023, 82, 100)
y shape: (3023,)


In [30]:
from sklearn.model_selection import train_test_split
X_train1,X_test1,Y_train,Y_test = train_test_split(X1,y,test_size=0.1,random_state=10)
X_train2,X_test2,Y_train,Y_test = train_test_split(X2,y,test_size=0.1,random_state=10)
X_train3,X_test3,Y_train,Y_test = train_test_split(X3,y,test_size=0.1,random_state=10)
X_train4,X_test4,Y_train,Y_test = train_test_split(X4,y,test_size=0.1,random_state=10)
#shape of train and test objects
print(X_train1.shape)
print(X_test1.shape)
# shape of new y objects
print(Y_train.shape)
print(Y_test.shape)

(2720, 82, 100)
(303, 82, 100)
(2720,)
(303,)


In [33]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.layers import Input, Embedding, Convolution1D, Dense, Dropout, Flatten, Conv1D, concatenate
from keras.optimizers import Adam

inputA = Input(shape=(82,100)) # One
inputB = Input(shape=(81,100)) # two
inputC = Input(shape=(80,100)) # Three
inputD = Input(shape=(79,100)) # Four

a = Convolution1D(128, kernel_size=3, activation='relu', padding='same') (inputA)
a = Dropout(0.5) (a)
a = Convolution1D(64, kernel_size=3, activation='relu', padding='same') (a)
a = Dropout(0.5) (a)
a = Convolution1D(32, kernel_size=3, activation='relu', padding='same') (a)
a = Dropout(0.5) (a)
a = Flatten() (a)

b = Convolution1D(filters = 128, kernel_size= 3, activation='relu', padding='same') (inputB)
b = Dropout(0.50) (b)
b = Convolution1D(filters = 64, kernel_size = 3, activation='relu', padding='same') (b)
b = Dropout(0.50) (b)
b = Convolution1D(filters = 32, kernel_size = 3, activation='relu', padding='same') (b)
b = Dropout(0.50) (b)
b = Flatten() (b)

c = Convolution1D(256, kernel_size=3, activation='relu', padding='same') (inputC)
c = Dropout(0.5) (c)
c = Conv1D(128, kernel_size=3, activation='relu', padding='same') (c)
c = Dropout(0.5) (c)
c = Conv1D(64, kernel_size=3, activation='relu', padding='same') (c)
c = Dropout(0.5) (c)
c = Conv1D(32, kernel_size=3, activation='relu', padding='same') (c)
c = Dropout(0.5) (c)
c = Flatten() (c)

d = Convolution1D(128, kernel_size=3, activation='relu', padding='same') (inputD)
d = Dropout(0.5) (d)
d = Conv1D(64, kernel_size=3, activation='relu', padding='same') (d)
d = Dropout(0.5) (d)
d = Conv1D(32, kernel_size=3, activation='relu', padding='same') (d)
d = Dropout(0.5) (d)
d = Flatten() (d)

combined = concatenate([a, b, c, d])

h = Dropout(0.5) (combined)
h = Dense(128, activation='relu') (h)
h = Dropout(0.5) (h)
h = Dense(64, activation='relu') (h)
output = Dense(1, activation='sigmoid') (h)

model = Model(inputs=[inputA, inputB, inputC, inputD], outputs=output)
model.compile(Adam(lr=0.0003), loss='binary_crossentropy', metrics=['accuracy'])
model.fit([X_train1, X_train2, X_train3, X_train4], Y_train, validation_data=([X_test1, X_test2, X_test3, X_test4], Y_test),
          epochs=200, batch_size=32, verbose=1)

Train on 2720 samples, validate on 303 samples
Epoch 1/200
2720/2720 [==============================] - 3s 992us/step - loss: 0.7738 - accuracy: 0.5279 - val_loss: 0.6887 - val_accuracy: 0.5479
Epoch 2/200
2720/2720 [==============================] - 2s 594us/step - loss: 0.7130 - accuracy: 0.5243 - val_loss: 0.6887 - val_accuracy: 0.5644
Epoch 3/200
2720/2720 [==============================] - 2s 596us/step - loss: 0.7124 - accuracy: 0.5272 - val_loss: 0.6892 - val_accuracy: 0.5578
Epoch 4/200
2720/2720 [==============================] - 2s 582us/step - loss: 0.6976 - accuracy: 0.5449 - val_loss: 0.6836 - val_accuracy: 0.5710
Epoch 5/200
2720/2720 [==============================] - 2s 582us/step - loss: 0.6931 - accuracy: 0.5599 - val_loss: 0.6731 - val_accuracy: 0.6139
Epoch 6/200
2720/2720 [==============================] - 2s 583us/step - loss: 0.6790 - accuracy: 0.5710 - val_loss: 0.6344 - val_accuracy: 0.6205
Epoch 7/200
2720/2720 [==============================] - 2s 578us/step 

# **K- Fold**

In [0]:
yt = []
yp = []
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True)
for train_idx, val_idx in kf.split(X1):
  X_train1 = X1[train_idx]
  X_train2 = X2[train_idx]
  X_train3 = X3[train_idx]
  X_train4 = X4[train_idx]
  y_train = y[train_idx]

  X_val1 = X1[val_idx]
  X_val2 = X2[val_idx]
  X_val3 = X3[val_idx]
  X_val4 = X4[val_idx]
  y_val = y[val_idx]

  inputA = Input(shape=(81,100)) # One
  inputB = Input(shape=(80,100)) # two
  inputC = Input(shape=(79,100)) # Three
  inputD = Input(shape=(78,100)) # Four

  a = Convolution1D(128, kernel_size=3, activation='relu', padding='same') (inputA)
  a = Dropout(0.5) (a)
  a = Convolution1D(64, kernel_size=3, activation='relu', padding='same') (a)
  a = Dropout(0.5) (a)
  a = Convolution1D(32, kernel_size=3, activation='relu', padding='same') (a)
  a = Dropout(0.5) (a)
  a = Flatten() (a)

  b = Convolution1D(filters = 128, kernel_size= 3, activation='relu', padding='same') (inputB)
  b = Dropout(0.50) (b)
  b = Convolution1D(filters = 64, kernel_size = 3, activation='relu', padding='same') (b)
  b = Dropout(0.50) (b)
  b = Convolution1D(filters = 32, kernel_size = 3, activation='relu', padding='same') (b)
  b = Dropout(0.50) (b)
  b = Flatten() (b)

  c = Convolution1D(256, kernel_size=3, activation='relu', padding='same') (inputC)
  c = Dropout(0.5) (c)
  c = Conv1D(128, kernel_size=3, activation='relu', padding='same') (c)
  c = Dropout(0.5) (c)
  c = Conv1D(64, kernel_size=3, activation='relu', padding='same') (c)
  c = Dropout(0.5) (c)
  c = Conv1D(32, kernel_size=3, activation='relu', padding='same') (c)
  c = Dropout(0.5) (c)
  c = Flatten() (c)

  d = Convolution1D(128, kernel_size=3, activation='relu', padding='same') (inputD)
  d = Dropout(0.5) (d)
  d = Conv1D(64, kernel_size=3, activation='relu', padding='same') (d)
  d = Dropout(0.5) (d)
  d = Conv1D(32, kernel_size=3, activation='relu', padding='same') (d)
  d = Dropout(0.5) (d)
  d = Flatten() (d)

  combined = concatenate([a, b, c, d])

  h = Dropout(0.5) (combined)
  h = Dense(128, activation='relu') (h)
  h = Dropout(0.5) (h)
  h = Dense(64, activation='relu') (h)
  output = Dense(1, activation='sigmoid') (h)

  model = Model(inputs=[inputA, inputB, inputC, inputD], outputs=output)
  model.compile(Adam(lr=0.0003), loss='binary_crossentropy', metrics=['accuracy'])
  model.fit([X_train1, X_train2, X_train3, X_train4], y_train, 
            validation_data=([X_val1, X_val2, X_val3, X_val4], y_val),
            epochs=10, batch_size=32, verbose=1)


  probabilities = model.predict([X_val1, X_val2, X_val3, X_val4])
  predicted_classes = probabilities >= 0.5
  predicted_classes = predicted_classes.astype(int)
  yp.append(predicted_classes)
  yt.append(y_val)

Train on 4576 samples, validate on 1144 samples
Epoch 1/10
4576/4576 [==============================] - 3s 713us/step - loss: 0.7719 - accuracy: 0.4952 - val_loss: 0.6884 - val_accuracy: 0.6110
Epoch 2/10
4576/4576 [==============================] - 2s 509us/step - loss: 0.6825 - accuracy: 0.5610 - val_loss: 0.5767 - val_accuracy: 0.7351
Epoch 3/10
4576/4576 [==============================] - 2s 503us/step - loss: 0.5564 - accuracy: 0.7310 - val_loss: 0.4884 - val_accuracy: 0.7736
Epoch 4/10
4576/4576 [==============================] - 2s 506us/step - loss: 0.5256 - accuracy: 0.7476 - val_loss: 0.4966 - val_accuracy: 0.7517
Epoch 5/10
4576/4576 [==============================] - 2s 502us/step - loss: 0.5211 - accuracy: 0.7509 - val_loss: 0.4778 - val_accuracy: 0.7841
Epoch 6/10
4576/4576 [==============================] - 2s 519us/step - loss: 0.4975 - accuracy: 0.7677 - val_loss: 0.4696 - val_accuracy: 0.7876
Epoch 7/10
4576/4576 [==============================] - 2s 506us/step - loss

In [0]:
true = []
pred = []
a=0
for i in yt:
  b=0
  for j in i:
    true.append(j)
    pred.append(yp[a][b])
    b=+1
  a+=1
    
y_true = np.asarray(true)
y_pred = np.asarray(pred)

print('Acc: ', accuracy_score(y_true, y_pred) * 100)
print('MCC: ', matthews_corrcoef(y_true, y_pred) * 100)

cm = confusion_matrix(y_true, y_pred)
print('Sn:  ', (cm[1,1] / (cm[1,1] + cm[1,0])) * 100)
print('Sp:  ', (cm[0,0] / (cm[0,0] + cm[0,1])) * 100)

p = precision_recall_fscore_support(y_true, y_pred)
print('pre_ 0: ', p[0][0] * 100)
print('pre_ 1: ', p[0][1] * 100)
print('Pre_weighted: ', precision_score(y_true, y_pred, average='weighted') * 100)

print('F-score: ', np.mean(p[2]))